In [3]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [4]:
env = GridworldEnv()

In [16]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation one our value function change is less than theta for all states.
        discount_factor: lambda discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """

    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    last_v = V + theta + 100
    while True:
        for s in range(env.nS):
            # Calculate the next V[s] (synchronous backup)
            v = 0
            # For each possible action
            for a, a_prob in enumerate(policy[s]):
                # Find each possible next_state
                for (prob, next_state, reward, done) in env.P[s][a]:
                    v += a_prob * prob * (reward + discount_factor * V[next_state])
            V[s] = v
        if (np.absolute(V - last_v) < theta).all():
            break
        last_v = np.array(V)
    return np.array(V)

In [17]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [18]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)